In [1]:
import torch
print("🚀 Using device:", "cuda" if torch.cuda.is_available() else "cpu")

🚀 Using device: cuda


In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

2.8.0+cu128
True
NVIDIA H100 80GB HBM3


### Create a HF dataset

In [3]:
import os, json, re
from pathlib import Path
from tqdm import tqdm
import soundfile as sf
from datasets import load_dataset, Audio as HF_Audio

OUT_ROOT = Path("/mnt/ebs_volume/HF_AZ")
OUT_ROOT.mkdir(parents=True, exist_ok=True)

ds = load_dataset("LocalDoc/azerbaijani_asr", split="train", streaming=True)
ds = ds.cast_column("audio", HF_Audio(sampling_rate=16_000))

def safe_slug(s: str) -> str:
    import re
    s = s.strip().replace(" ", "_")
    return re.sub(r"[^0-9A-Za-z_\-\.]+", "_", s)[:120]

written = 0
for i, ex in tqdm(enumerate(ds), desc="Exporting HF_AZ (streaming)"):
    audio = ex["audio"]
    text  = (ex.get("text") or "").strip()
    src   = ex.get("audio_file") or f"hf_{i:09d}"

    base = f"hfaz_{i:09d}__{safe_slug(Path(src).stem)}"
    wav_path = OUT_ROOT / f"{base}.wav"
    txt_path = OUT_ROOT / f"{base}.wav.txt"

    if not text:
        continue

    if not wav_path.exists():
        sf.write(str(wav_path), audio["array"], audio["sampling_rate"], subtype="PCM_16")

    with open(txt_path, "w", encoding="utf-8") as f:
        json.dump({"text": text, "audio_path": str(wav_path)}, f, ensure_ascii=False)

    written += 1

print("Done. Wrote", written, "pairs to", OUT_ROOT)

Resolving data files:   0%|          | 0/133 [00:00<?, ?it/s]

Exporting HF_AZ (streaming): 351019it [1:04:54, 90.14it/s] 

Done. Wrote 351019 pairs to /mnt/ebs_volume/HF_AZ


In [4]:
import os, json
from glob import glob
from datasets import Dataset, Audio as HF_Audio

ROOTS = ["/mnt/ebs_volume/QURBAN1",
         "/mnt/ebs_volume/NIJAT2",
         "/mnt/ebs_volume/NIJAT",
         "/mnt/ebs_volume/GASIM",
         "/mnt/ebs_volume/HF_AZ",
]

SMOKE_TEST = False   # set False for full 911k run
SMOKE_MAX  = 500    # how many pairs to load in smoke test

def collect_pairs(roots, smoke=False, k=500):
    pairs = []
    for root in roots:
        for wav_path in glob(os.path.join(root, "**", "*.wav"), recursive=True):
            txt_path = wav_path + ".txt"
            if os.path.exists(txt_path):
                pairs.append((wav_path, txt_path))
    pairs = sorted(pairs)
    return pairs[:k] if smoke else pairs

pairs = collect_pairs(ROOTS, smoke=SMOKE_TEST, k=SMOKE_MAX)
print(f"Found {len(pairs)} pairs ({'SMOKE' if SMOKE_TEST else 'FULL'})")

records = []
for wav_path, txt_path in pairs:
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        raw = f.read().strip()
    try:
        text = (json.loads(raw).get("text") or "").strip()
    except Exception:
        text = raw.strip()

    # simple hygiene filter (tune thresholds as you like)
    if len(text) < 2:
        continue

    records.append({"audio": wav_path, "text": text})

ds = Dataset.from_list(records)
ds = ds.cast_column("audio", HF_Audio(sampling_rate=16_000))

save_dir = "./transcription_pairs_16k_sample" if SMOKE_TEST else "./transcription_pairs_16k_full"
ds.save_to_disk(save_dir)
print(ds)
print("Saved to", save_dir)

Found 1265847 pairs (FULL)


Saving the dataset (0/217 shards):   0%|          | 0/1265840 [00:00<?, ? examples/s]

Dataset({
    features: ['audio', 'text'],
    num_rows: 1265840
})
Saved to ./transcription_pairs_16k_full


In [2]:
import os
os.environ["HF_DATASETS_CACHE"] = "/mnt/ebs_volume/hf_cache"   # datasets cache
os.environ["TRANSFORMERS_CACHE"] = "/mnt/ebs_volume/hf_models" # (optional) model weights
os.environ["HF_HOME"] = "/mnt/ebs_volume/hf_home"              # (optional) central HF dir

print("HF_DATASETS_CACHE =", os.getenv("HF_DATASETS_CACHE"))
print("TRANSFORMERS_CACHE =", os.getenv("TRANSFORMERS_CACHE"))

HF_DATASETS_CACHE = /mnt/ebs_volume/hf_cache
TRANSFORMERS_CACHE = /mnt/ebs_volume/hf_models


In [13]:
import os, re, json, textwrap
from datasets import load_from_disk, Audio

DATA_DIR = "./transcription_pairs_16k_full"

# Load without decoding audio arrays (faster for text filtering)
ds = load_from_disk(DATA_DIR).cast_column("audio", Audio(sampling_rate=16_000, decode=False))

# 1) Normalize common confusables to AZ Latin before filtering
CONFUSABLE_MAP = {
    "\u04D8": "\u018F",  # Cyrillic Ә -> Latin Ə
    "\u04D9": "\u0259",  # Cyrillic ә -> Latin ə
    # (Add more if you discover others causing false drops)
}
CONFUSABLE_RE = re.compile("|".join(map(re.escape, CONFUSABLE_MAP.keys())))

def normalize_confusables(s: str) -> str:
    if not s:
        return s
    return CONFUSABLE_RE.sub(lambda m: CONFUSABLE_MAP[m.group(0)], s)

# 2) Strong negative filter for non-Latin scripts (keeps Latin + AZ letters)
NON_LATIN_RE = re.compile(
    r"[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF"   # Arabic blocks
    r"\u0590-\u05FF"                               # Hebrew
    r"\u0400-\u052F"                               # Cyrillic
    r"\u0370-\u03FF"                               # Greek
    r"\u10A0-\u10FF\u2D00-\u2D2F"                  # Georgian
    r"\u0530-\u058F]"                              # Armenian
)

def looks_azerbaijani_str(t: str) -> bool:
    t = normalize_confusables((t or "").strip())
    if len(t) < 2:
        return False
    if NON_LATIN_RE.search(t):
        return False
    # require at least one letter so we don't keep pure digits/punct
    return any(ch.isalpha() for ch in t)

def keep_mask(batch):
    return [looks_azerbaijani_str(t) for t in batch["text"]]

# --- QUICK PEEK again (optional): show first 5 that would now be rejected after normalization
N = 5
bad = []
for ex in ds:
    txt_norm = normalize_confusables(ex["text"])
    if not looks_azerbaijani_str(txt_norm):
        bad.append((ex["audio"]["path"], txt_norm))
        print(f"\n[{len(bad)}] {ex['audio']['path']}\n"
              f"{textwrap.shorten(txt_norm, width=300, placeholder=' …')}")
        if len(bad) >= N:
            break
if bad:
    print(f"\nStopped after collecting {len(bad)} rejected examples (post-normalization).")
else:
    print("No rejected examples found in the scanned portion.")

Loading dataset from disk:   0%|          | 0/217 [00:00<?, ?it/s]


[1] 222edf_utt_0007.wav
სასეიდე ტელეფონაჩაც.

[2] 222edf_utt_0016.wav
ოინეიდეინე, ჯეკ, გუგუშოდა!

[3] 222edf_utt_0076.wav
Баспома рахлоладжах.

[4] 222edf_utt_0140.wav
Усылығнан калибрек ушны көзеннəн борара.

[5] 222edf_utt_0141.wav
باچاخیله کرده.

Stopped after collecting 5 rejected examples (post-normalization).


In [14]:
# --- FULL FILTER + SAVE (original stays untouched)
filtered = ds.filter(keep_mask, batched=True, batch_size=10_000)
print(f"Kept {len(filtered):,} / {len(ds):,} ({len(filtered)/len(ds):.1%})")

out_path = "/mnt/ebs_volume/transcription_pairs_16k_filtered"
filtered.save_to_disk(out_path)
print("Saved filtered dataset to:", out_path)

Filter:   0%|          | 0/1265840 [00:00<?, ? examples/s]

Kept 1,236,203 / 1,265,840 (97.7%)


Saving the dataset (0/252 shards):   0%|          | 0/1236203 [00:00<?, ? examples/s]

Saved filtered dataset to: /mnt/ebs_volume/transcription_pairs_16k_filtered


## Fine-Tuning

### Precompute and save once the mapping

In [1]:
import os
os.environ["HF_DATASETS_CACHE"] = "/mnt/ebs_volume/hf_cache"   # datasets cache
os.environ["TRANSFORMERS_CACHE"] = "/mnt/ebs_volume/hf_models" # (optional) model weights
os.environ["HF_HOME"] = "/mnt/ebs_volume/hf_home"              # (optional) central HF dir

print("HF_DATASETS_CACHE =", os.getenv("HF_DATASETS_CACHE"))
print("TRANSFORMERS_CACHE =", os.getenv("TRANSFORMERS_CACHE"))

HF_DATASETS_CACHE = /mnt/ebs_volume/hf_cache
TRANSFORMERS_CACHE = /mnt/ebs_volume/hf_models


In [2]:
import os
import torch, evaluate
from datasets import load_from_disk, Audio
from transformers import (WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor,
                          WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer)
from dataclasses import dataclass
from typing import Any, Dict, List, Union

print("🚀 Using device:", "cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = "./transcription_pairs_16k_filtered"
ds = load_from_disk(DATA_DIR).cast_column("audio", Audio(sampling_rate=16_000))

MODEL_ID = "openai/whisper-large-v3"
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_ID)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_ID, language="Azerbaijani", task="transcribe")
processor = WhisperProcessor.from_pretrained(MODEL_ID, language="Azerbaijani", task="transcribe")

#def prepare_batch(batch):
 #   a = batch["audio"]
 #    feats = feature_extractor(a["array"], sampling_rate=a["sampling_rate"]).input_features[0]
 #    labels = tokenizer(batch["text"]).input_ids
 #    return {"input_features": feats, "labels": labels}

#proc = ds.map(prepare_batch, remove_columns=ds.column_names, num_proc=1)

def prepare_batch(batch):
    arrays = [a["array"] for a in batch["audio"]]
    feats  = feature_extractor(arrays, sampling_rate=16000).input_features
    labels = tokenizer(batch["text"]).input_ids
    return {"input_features": feats, "labels": labels}

proc = ds.map(
    prepare_batch,
    remove_columns=ds.column_names,
    batched=True,
    batch_size=64,                  # batch into groups to reduce Python overhead
    num_proc=os.cpu_count() // 2    # try half your CPU cores
)

/mnt/ebs_volume/jlab/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🚀 Using device: cuda


Loading dataset from disk:   0%|          | 0/252 [00:00<?, ?it/s]

In [1]:
import os
os.environ["HF_DATASETS_CACHE"] = "/mnt/ebs_volume/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/mnt/ebs_volume/hf_models"
os.environ["HF_HOME"] = "/mnt/ebs_volume/hf_home"

import torch, evaluate
from datasets import load_from_disk
from transformers import (
    WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor,
    WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union

print("🚀 Using device:", "cuda" if torch.cuda.is_available() else "cpu")

MODEL_ID = "openai/whisper-large-v3"
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_ID)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_ID, language="Azerbaijani", task="transcribe")
processor = WhisperProcessor.from_pretrained(MODEL_ID, language="Azerbaijani", task="transcribe")

dd = load_from_disk("/mnt/ebs_volume/transcription_pairs_16k_mapped_lenle448")
train_ds, eval_ds = dd["train"], dd["test"]  # ready to train immediately

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        inputs = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(inputs, return_tensors="pt")
        label_features = [{"input_ids": f["labels"]} for f in features]
        label_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = label_batch["input_ids"].masked_fill(label_batch.attention_mask.ne(1), -100)
        # if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
        #     labels = labels[:, 1:]
        # batch["labels"] = labels
        
        # drop leading BOS if present everywhere (Whisper tokenizer often prepends it)
        if labels.numel() > 0 and (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
wer_metric = evaluate.load("wer")

def compute_metrics(pred):
    pred_ids, label_ids = pred.predictions, pred.label_ids
    label_ids = label_ids.copy()
    label_ids[label_ids == -100] = tokenizer.pad_token_id
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    return {"wer": 100 * wer_metric.compute(predictions=pred_str, references=label_str)}

model = WhisperForConditionalGeneration.from_pretrained(MODEL_ID)
model.generation_config.language = "az"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []

# Prefer bf16 on Hopper (H100), else fp16 if CUDA available
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

SMOKE = False
out_dir = "/mnt/ebs_volume/whisper-az-largev3-smoke" if SMOKE else "/mnt/ebs_volume/whisper-az-largev3-finetuned"

args = (Seq2SeqTrainingArguments(
    output_dir=out_dir,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=4,
    learning_rate=1e-5,
    max_steps=50,                 # smoke test
    eval_strategy="steps",
    eval_steps=25,
    save_steps=25,
    logging_steps=10,
    fp16=True,                    # consider bf16=True on H100 if your env supports it
    predict_with_generate=True,
    generation_max_length=225,
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
) if SMOKE else Seq2SeqTrainingArguments(
    output_dir=out_dir,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=6,
    learning_rate=1e-5,
    num_train_epochs=5,
    eval_strategy="no",
    save_strategy="epoch",
    logging_steps=25,
    bf16=use_bf16,
    fp16=(not use_bf16) and torch.cuda.is_available(),
    predict_with_generate=False,
    generation_max_length=225,
    load_best_model_at_end=False,
    #metric_for_best_model="wer",
    #greater_is_better=False,
    dataloader_num_workers=4,         # try 4–8
    group_by_length=False,             # buckets by duration -> better throughput
    remove_unused_columns=False,
    push_to_hub=False,
))

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    #eval_dataset=eval_ds,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,
    #compute_metrics=compute_metrics,
)

processor.save_pretrained(args.output_dir)
trainer.train()

/mnt/ebs_volume/jlab/lib/python3.12/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


🚀 Using device: cuda


Loading dataset from disk:   0%|          | 0/3420 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/380 [00:00<?, ?it/s]

/tmp/ipykernel_3215544/1048083705.py:107: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__ca

Step,Training Loss
25,1.102200
50,0.759400
75,0.724900
100,0.597200
125,0.606200
150,0.579600
175,0.607100
200,0.491100
225,0.514400
250,0.531200


/mnt/ebs_volume/jlab/lib/python3.12/site-packages/transformers/modeling_utils.py:4037: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text

TrainOutput(global_step=463575, training_loss=0.18816368085463855, metrics={'train_runtime': 214311.9279, 'train_samples_per_second': 25.957, 'train_steps_per_second': 2.163, 'total_flos': 1.889980742963036e+22, 'train_loss': 0.18816368085463855, 'epoch': 5.0})

In [2]:
# quick sanity decode
import torch
sample = eval_ds.select(range(min(3, len(eval_ds))))
with torch.no_grad():
    feats = torch.tensor([x["input_features"] for x in sample], device=model.device)
    gen = model.generate(inputs=feats, max_length=225)
print("\nSample predictions:")
for i, s in enumerate(tokenizer.batch_decode(gen, skip_special_tokens=True)):
    print(f"[{i}] {s}")

/mnt/ebs_volume/jlab/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:655: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Sample predictions:
[0] Așșcalețeleibă.
[1] Hər dəfə uşaqla bazara gedəndə,
[2] Yepişməci hallarının qarşısını almağa kömək edir. Molfix ana qucağı kimi rahat.


In [1]:
import os, re, time, numpy as np, torch, evaluate, glob
from datasets import load_from_disk
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments
from dataclasses import dataclass
from typing import Any, Dict, List, Union

OUT_DIR = "/mnt/ebs_volume/whisper-az-largev3-finetuned"   # <-- your training output_dir
CKPTS = sorted(glob.glob(os.path.join(OUT_DIR, "checkpoint-*")),
               key=lambda p: int(re.findall(r"checkpoint-(\d+)", p)[0]))

print("Found checkpoints:", [os.path.basename(p) for p in CKPTS])

# Load eval split you already saved
dd = load_from_disk("/mnt/ebs_volume/transcription_pairs_16k_mapped_lenle448")
eval_ds_full = dd["test"]

# For speed, subsample (tweak this)
MAX_EVAL = 500
eval_ds = eval_ds_full.shuffle(seed=0).select(range(min(MAX_EVAL, len(eval_ds_full))))

# Rebuild your collator (same as training)
from transformers import WhisperProcessor, WhisperTokenizer, WhisperFeatureExtractor
MODEL_ID = "openai/whisper-large-v3"
feature_extractor = WhisperFeatureExtractor.from_pretrained(MODEL_ID)
tokenizer = WhisperTokenizer.from_pretrained(MODEL_ID, language="Azerbaijani", task="transcribe")
processor = WhisperProcessor.from_pretrained(MODEL_ID, language="Azerbaijani", task="transcribe")

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        inputs = [{"input_features": f["input_features"]} for f in features]
        batch = self.processor.feature_extractor.pad(inputs, return_tensors="pt")
        label_features = [{"input_ids": f["labels"]} for f in features]
        label_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")
        labels = label_batch["input_ids"].masked_fill(label_batch.attention_mask.ne(1), -100)
        if labels.numel() > 0 and (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]
        batch["labels"] = labels
        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)
wer_metric = evaluate.load("wer")
use_bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

Found checkpoints: ['checkpoint-92715', 'checkpoint-185430', 'checkpoint-278145', 'checkpoint-370860', 'checkpoint-463575']


Loading dataset from disk:   0%|          | 0/3420 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/380 [00:00<?, ?it/s]

In [2]:
def _normalize_logits(pred_predictions):
    """Return logits as either a single ndarray (B, T, V) or list of (Ti, V)."""
    logits = pred_predictions
    if isinstance(logits, tuple):
        logits = logits[0]
    return logits

def compute_metrics_argmax(pred):
    logits = _normalize_logits(pred.predictions)
    label_ids = pred.label_ids

    # Replace -100 with pad for decoding refs
    label_ids = label_ids.copy()
    label_ids[label_ids == tokenizer.pad_token_id] = tokenizer.pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # Handle ragged vs dense logits
    if isinstance(logits, list):
        # per-example argmax then decode
        pred_ids = [np.argmax(x, axis=-1) for x in logits]  # list of (Ti,)
    else:
        pred_ids = np.argmax(logits, axis=-1)               # (B, T)

    # batch_decode works with list-of-lists or ndarray
    pred_str  = tokenizer.batch_decode(pred_ids,  skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    return {"wer": 100 * wer_metric.compute(predictions=pred_str, references=label_str)}

def eval_ckpt_argmax(ckpt_path):
    model = WhisperForConditionalGeneration.from_pretrained(
        ckpt_path, dtype=(torch.bfloat16 if use_bf16 else torch.float16)
    ).to("cuda")
    gc = model.generation_config
    gc.language = "az"; gc.task = "transcribe"
    gc.forced_decoder_ids = None; gc.suppress_tokens = []

    args = Seq2SeqTrainingArguments(
        output_dir="/tmp/eval",
        per_device_eval_batch_size=24,
        dataloader_num_workers=4,
        bf16=use_bf16, fp16=(not use_bf16) and torch.cuda.is_available(),
        predict_with_generate=False,
        remove_unused_columns=False,
        report_to="none",
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        eval_dataset=eval_ds,
        data_collator=data_collator,
        processing_class=processor,   # <- instead of tokenizer=...
        compute_metrics=compute_metrics_argmax,
    )
    t0 = time.time()
    metrics = trainer.evaluate()
    secs = time.time() - t0
    del trainer, model; torch.cuda.empty_cache()
    return metrics["eval_wer"], secs

In [3]:
eval_fn = eval_ckpt_argmax   # or eval_ckpt_generate

results = []
for ck in CKPTS:
    wer, secs = eval_fn(ck)
    results.append((ck, wer, secs))
    print(f"{os.path.basename(ck):>20}  WER={wer:.2f}%  time={secs/60:.1f} min")

# Sort & show best
results.sort(key=lambda x: x[1])  # lower WER is better
best_ck, best_wer, best_secs = results[0]
print("\nBEST:", os.path.basename(best_ck), f"WER={best_wer:.2f}%")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


    checkpoint-92715  WER=35.92%  time=1.0 min


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


...

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   checkpoint-370860  WER=35.74%  time=1.0 min


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a WhisperTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   checkpoint-463575  WER=39.28%  time=1.0 min

BEST: checkpoint-278145 WER=33.32%


In [ ]:
# app.py
import os
import time
import torch
import numpy as np
import gradio as gr

# Audio I/O
import soundfile as sf
import librosa

from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
)

# -----------------------------
# Paths
# -----------------------------
# If you exported to a single folder, set MODEL_DIR to that folder.
# Otherwise, set it to your run root that also contains tokenizer/preprocessor files.
MODEL_DIR = r"/mnt/ebs_volume/whisper-az-largev3-finetuned/checkpoint-278145"  # or ".../export-278145"

# -----------------------------
# Load model & processor
# -----------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
use_bf16 = (device == "cuda") and torch.cuda.get_device_capability(0)[0] >= 8  # Hopper+
dtype = torch.bfloat16 if use_bf16 else torch.float16 if device == "cuda" else torch.float32

processor = WhisperProcessor.from_pretrained(MODEL_DIR)
model = WhisperForConditionalGeneration.from_pretrained(MODEL_DIR, torch_dtype=dtype)
model.to(device)
model.eval()

# Make sure generation settings are right for AZ transcription
model.generation_config.language = "az"
model.generation_config.task = "transcribe"
model.generation_config.forced_decoder_ids = None
model.generation_config.suppress_tokens = []

SR = 16000  # Whisper expects 16kHz features

def _load_audio_from_path(path: str, target_sr: int = SR) -> np.ndarray:
    """Load any audio file path -> float32 mono @ target_sr."""
    # soundfile preserves native sr; librosa resamples
    wav, sr = sf.read(path, always_2d=False)
    if wav.ndim == 2:
        wav = wav.mean(axis=1)  # mono
    if sr != target_sr:
        wav = librosa.resample(wav, orig_sr=sr, target_sr=target_sr)
    wav = wav.astype(np.float32)
    # Clamp extreme amplitudes just in case
    maxabs = np.max(np.abs(wav)) if wav.size else 0.0
    if maxabs > 1.0:
        wav = wav / maxabs
    return wav

def _normalize_input(audio):
    """
    Accepts Gradio Audio in v3 (tuple (sr, np.ndarray)) or v4 (np.ndarray with sample_rate attr)
    or dict for file-based upload. Returns float32 mono at 16k.
    """
    # mic input (numpy)
    if isinstance(audio, tuple) and len(audio) == 2:
        sr, wav = audio
        if wav.ndim == 2:
            wav = wav.mean(axis=1)
        if sr != SR:
            wav = librosa.resample(wav, orig_sr=sr, target_sr=SR)
        return wav.astype(np.float32)

    # gradio v4 numpy with .sample_rate (fallback)
    if hasattr(audio, "shape") and hasattr(audio, "sample_rate"):
        wav = audio
        sr = getattr(audio, "sample_rate", SR)
        if wav.ndim == 2:
            wav = wav.mean(axis=1)
        if sr != SR:
            wav = librosa.resample(wav, orig_sr=sr, target_sr=SR)
        return wav.astype(np.float32)

    # file upload: gradio passes a dict like {"name":..., "size":..., "mime_type":..., "path":...}
    if isinstance(audio, dict) and "path" in audio:
        return _load_audio_from_path(audio["path"], target_sr=SR)

    # raw string path
    if isinstance(audio, str) and os.path.exists(audio):
        return _load_audio_from_path(audio, target_sr=SR)

    raise ValueError("Unsupported audio input format")

@torch.inference_mode()
def transcribe(audio_in):
    try:
        t0 = time.time()
        wav = _normalize_input(audio_in)
        if wav.size == 0:
            return "Empty audio."

        # Feature extraction (log-Mel) via processor
        feats = processor.feature_extractor(
            wav, sampling_rate=SR, return_tensors="pt"
        ).input_features  # [1, 80, T]
        feats = feats.to(device)

        # Generate (greedy by default; tune max_length if needed)
        with torch.autocast(device_type="cuda", dtype=dtype) if device == "cuda" else torch.no_grad():
            out_ids = model.generate(
                inputs=feats,
                max_length=225,           # consistent with your training
                do_sample=False,
            )

        text = processor.tokenizer.batch_decode(out_ids, skip_special_tokens=True)[0]
        elapsed = time.time() - t0
        return f"{text}\n\n⏱️ {elapsed:.2f}s"
    except Exception as e:
        return f"Error: {e}"

# -----------------------------
# Gradio UI
# -----------------------------
with gr.Blocks(title="Whisper AZ Transcriber") as demo:
    gr.Markdown("## Whisper AZ Transcriber\nRecord or upload audio; model returns the transcript.")

    with gr.Row():
        audio = gr.Audio(
            sources=["microphone", "upload"],
            type="numpy",            # gives (sr, numpy) for mic; file uploads return dict
            label="Microphone or upload (.wav/.mp3/etc.)",
        )
    btn = gr.Button("Transcribe")
    out = gr.Textbox(label="Transcript", lines=6)

    btn.click(transcribe, inputs=audio, outputs=out)

if __name__ == "__main__":
    # Launch on all interfaces so you can open it from your laptop browser via the server IP
    demo.launch(server_name="0.0.0.0", server_port=7860)


In [ ]:
trainer.save_model(args.output_dir)         # ensure final model lands in out_dir
processor.save_pretrained(args.output_dir)  # ensure processor is alongside the model

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
model = WhisperForConditionalGeneration.from_pretrained("/path/to/out_dir")
processor = WhisperProcessor.from_pretrained("/path/to/out_dir")

In [ ]:
# save to disk
#proc.save_to_disk("/mnt/ebs_volume/transcription_pairs_16k_mapped")